# Install the required dependencies:


In [2]:
!pip install cassio datasets langchain tiktoken langchain-huggingface langchain-google-genai langchain-community


# Import the packages you'll need

In [3]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings

from datasets import load_dataset

import cassio

In [6]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.2 MB/s eta 0:00:00


In [7]:
from PyPDF2 import PdfReader

In [19]:
ASTRA_DB_APPLICATION_TOKEN = ""
ASTRA_DB_ID = ""

GEMINI_API_KEY = ""

In [11]:
pdfReader = PdfReader("/content/MSMED2006act.pdf")

In [12]:
from typing_extensions import Concatenate
#read text from pdf
raw_text = ''
for i, page in enumerate(pdfReader.pages):
  content = page.extract_text()
  if content:
    raw_text += content


In [20]:
raw_text

"1 \n THE MICRO, SMALL AND MEDIUM ENTERPRISES DEVELOPMENT ACT, 2006  \n__________________  \n \nARRANGEMENT OF SECTIONS  \n__________________  \n \nCHAPTER I  \nPRELIMINARY  \nSECTIONS  \n1. Short title and commencement.  \n2. Definitions.  \n \nCHAPTER II  \nNATIONAL BOARD FOR MICRO, SMALL AND MEDIUM ENTERPRISES  \n3. Establishment of Board.  \n4. Removal of member from Board.  \n5. Functions of Board.  \n6. Powers and functions of Member -Secretary of Board.  \n \nCHAPTER III  \nCLASSIFICATION OF ENTERPRISES, ADVISORY COMMITTEE ANDMEMORANDUM  \nOF MICRO, SMALL AND MEDIUM ENTERPRISES  \n7. Classification of enterprises.  \n8. Memorandum of micro, small and m edium enterprises.  \n \nCHAPTER IV  \nMEASURES FOR PROMOTION, DEVELOPMENT AND ENHANCEMENT OFCOMPETITIVENESS OF MICRO,  \n SMALL AND MEDIUM ENTERPRISES  \n9. Measures for promotion and development.  \n10. Credit facil ities.  \n11. Procurement preference policy.  \n12. Funds.  \n13. Grants by Central Government.  \n14. Administrat

In [ ]:
cassio.init(
    token="ASTRA_DB_APPLICATION_TOKEN",
    database_id="ASTRA_DB_ID",
)


In [16]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
astra_vector_store = Cassandra(
    embedding = embedding,
    table_name = "qa_mini_demo",
    session=None,
    keyspace=None,
)

In [24]:
from langchain import text_splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [26]:
texts[:50]

['1 \n THE MICRO, SMALL AND MEDIUM ENTERPRISES DEVELOPMENT ACT, 2006  \n__________________  \n \nARRANGEMENT OF SECTIONS  \n__________________  \n \nCHAPTER I  \nPRELIMINARY  \nSECTIONS  \n1. Short title and commencement.  \n2. Definitions.  \n \nCHAPTER II  \nNATIONAL BOARD FOR MICRO, SMALL AND MEDIUM ENTERPRISES  \n3. Establishment of Board.  \n4. Removal of member from Board.  \n5. Functions of Board.  \n6. Powers and functions of Member -Secretary of Board.  \n \nCHAPTER III  \nCLASSIFICATION OF ENTERPRISES, ADVISORY COMMITTEE ANDMEMORANDUM  \nOF MICRO, SMALL AND MEDIUM ENTERPRISES  \n7. Classification of enterprises.  \n8. Memorandum of micro, small and m edium enterprises.  \n \nCHAPTER IV  \nMEASURES FOR PROMOTION, DEVELOPMENT AND ENHANCEMENT OFCOMPETITIVENESS OF MICRO,  \n SMALL AND MEDIUM ENTERPRISES',
 '8. Memorandum of micro, small and m edium enterprises.  \n \nCHAPTER IV  \nMEASURES FOR PROMOTION, DEVELOPMENT AND ENHANCEMENT OFCOMPETITIVENESS OF MICRO,  \n SMALL AND MEDIUM

In [ ]:
from langchain_core import vectorstores
astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore = astra_vector_store)

In [ ]:
first_question = True
while True:
  if first_question:
    query_text = input("\Enter your question (or type 'quit' to exit): ").strip()
  else:
    query_text = input("\What's your next question (or type 'quit' to exit): ").strip()

  if query_text.lower() == "quit":
    break

  if query_text == "":
    continue

  first_question = False

  print("\nQuestion: \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm = llm)
  print("\nAnswer: \"%s\"\n" % answer)

  print("FIRST DOCUMENTS BY RELEVANCE:")
  for doc in astra_vector_store.similarity_search(query_text, k = 3):
    print("     [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))
